In [1]:
# !pip install telepot
# !pip install telepot --upgrade
# !pip install pyTelegramBotAPI

In [2]:
import codecs
def getNews():
    nfile = codecs.open('News.txt','r',encoding='utf8')     
    return u''.join(nfile)

In [3]:
from datetime import datetime
from googleCalendar import getShedule, create_calendar, getWeeklyShedule
def processShedule(data):
    if data[0] == 'getSheduleToday':
        return getWeeklyShedule()
    elif data[0] == 'getSheduleTommoroe':
        return getWeeklyShedule(offset = 1)
    elif data[0] == 'getSheduleWeekly':
        return getWeeklyShedule(week = True)
    elif data[0] == 'getSheduleNextWeekly':
        return getWeeklyShedule(week = True, offset = 7)
    elif data[0] == 'getSheduleDay':
        return getWeeklyShedule(day = map(int, data[1:]))

def createMarkup(keys):
    buttons = []
    for row in keys:
        rbuttons = []
        for k in row:
            callback = 'ignore' if len(k) == 1 else k[1]
            rbuttons.append(InlineKeyboardButton(text = k[0], callback_data = callback))
        buttons.append(rbuttons)
    return InlineKeyboardMarkup(inline_keyboard=buttons)

def createCalendar(chat_id, date = None):
    if date is None:
        now = datetime.now()
        return createMarkup(create_calendar(now.year,now.month))
    return createMarkup(create_calendar(date / 12, date % 12 + 1))

In [ ]:
def addNews(news):
    nfile = codecs.open('News.txt','a',encoding='utf8')     

In [8]:
def handleImpl(msg):
    print msg
    content_type, chat_type, chat_id = telepot.glance(msg)
    request = msg['text'].split(' ')
    print 'handleImpl', request[0]
    if request[0] == '/schedule':
        markup = createCalendar(chat_id)
        bot.sendMessage(chat_id, 'Выбирите день', reply_markup=markup)
        
    elif request[0] == '/news':
        bot.sendMessage(chat_id, getNews())
    elif request[0] == '/addNews' and msg['from']['username'] == 'djack239':
        addNews(request[1:])

def callbackImpl(msg):
    query_id, chat_id, query_data = telepot.glance(msg, flavor='callback_query')
    data = query_data.split(' ')
    print 'callbackImpl, data', data
    shed = processShedule(data)
    if shed:
        bot.sendMessage(chat_id, u'\n'.join(shed))
        return
    if data[0] == 'calendar' and len(data) > 1:
        try:
            date = int(data[1])
            markup = createCalendar(chat_id, date)
            bot.sendMessage(chat_id, 'Выбирите день', reply_markup=markup)
        except:
            pass

def choseImpl(msg):
    print 'choseImpl', msg

{u'from': {u'username': u'djack239', u'first_name': u'Evgeny', u'last_name': u'Demenkov', u'is_bot': False, u'language_code': u'ru', u'id': 370136783}, u'text': u'/addNews', u'entities': [{u'length': 8, u'type': u'bot_command', u'offset': 0}], u'chat': {u'username': u'djack239', u'first_name': u'Evgeny', u'last_name': u'Demenkov', u'type': u'private', u'id': 370136783}, u'date': 1509528754, u'message_id': 541}
handleImpl /addNews


In [5]:
def handle(msg):
    return handleImpl(msg)
def callback(msg):
    return callbackImpl(msg)
def chose(msg):
    return choseImpl(msg)

from telepot.loop import MessageLoop
import telepot
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton, ReplyKeyboardMarkup, KeyboardButton
token = next(open('token.txt'))
bot = telepot.Bot(token)
MessageLoop(bot, {'chat': handle,
                  'callback_query': callback,
                  'chosen_inline_result': chose}).run_as_thread()

In [6]:
import locale
# locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

In [7]:
1

1